<a href="https://colab.research.google.com/github/macgyver121/Project-Chart-to-Text-generation/blob/main/Final_modelT5base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check GPU

In [ ]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5f73edc7-1a58-dae2-c05a-56558d0f6fa1)


In [ ]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


## Install pytesseract

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,278 kB/s)
debconf: unable to initi

     |████████████████████████████████| 5.8 MB 28.9 MB/s 
     |████████████████████████████████| 452 kB 79.3 MB/s 
     |████████████████████████████████| 7.6 MB 73.3 MB/s 
     |████████████████████████████████| 182 kB 82.2 MB/s 
     |████████████████████████████████| 213 kB 83.1 MB/s 
     |████████████████████████████████| 132 kB 82.6 MB/s 
     |████████████████████████████████| 127 kB 62.0 MB/s 
     |████████████████████████████████| 800 kB 34.2 MB/s 
     |████████████████████████████████| 1.9 MB 67.1 MB/s 
     |████████████████████████████████| 125 kB 80.5 MB/s 
     |████████████████████████████████| 512 kB 54.3 MB/s 
     |████████████████████████████████| 184 kB 75.0 MB/s 
     |████████████████████████████████| 174 kB 76.7 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 173 kB 79.0 MB/s 
     |████████████████████████████████| 168 kB 79.1 MB/s 
     |████████████████████████████████| 168 kB 79.3 MB/s 
     |██████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

## Import library

In [ ]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Model1 for Chart2text dataset

### Create Data from Chart2text data

In [ ]:
List_key = []
# Open the CSV file for reading
for i in range(2000,4000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

2000

In [ ]:
import pickle

with open('List_key_2000to4000.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load

In [ ]:
import pickle

with open('List_key_6000_6763.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [ ]:
len(List_key)

763

In [ ]:
txt_key = []
for i in range(2000,4000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

2000

In [ ]:
import pickle

with open('txt_key_2000to4000.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load

In [ ]:
import pickle

with open('txt_key_6000_6763.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [ ]:
print(List_key[0])

Year Price index in real 2010 U.S. dollars 2030 87.2 2025 79.1 2022 74.7 2021 73.3 2020 72.0 2019 74.3 2018 87.0 2017 68.1 2016 55.1 2015 65.0 2014 111.7 2013 120.1


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Create a list of lists that represents the data
train_data = [List_key[:1400] ,txt_key[:1400]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[1400:1700] ,txt_key[1400:1700]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[1700:] ,txt_key[1700:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 1400
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 300
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 300
})}
<class 'dict'>


In [ ]:
# for 763 data
# Create a list of lists that represents the data
train_data = [List_key[:533] ,txt_key[:533]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[533:648] ,txt_key[533:648]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[648:] ,txt_key[648:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 533
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 115
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 115
})}
<class 'dict'>


In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 533
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 115
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 115
    })
})


In [ ]:
print(dataset['train'][1])

{'key': 'Brand Share of respondents Sherwin-Williams 49.5% Benjamin Moore 22.4% Behr Paint Cooperation 12.1% Kelly Moore 2.8% Valspar 1.9% PPG Pittsburgh Paints 1.9% Zar (United Gilsonite Labs) 1.9% Devoe & Raynolds 0.9% Dutch Boy 0.9% Olympic 0.9% None of these 4.7%', 'text': 'This statistic depicts paints used the most by U.S. construction firms in 2018 . The survey revealed that 49.5 percent of the respondents used Sherwin-Williams brand paints the most . '}


In [ ]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: Brand Share of respondents Sherwin-Williams 49.5% Benjamin Moore 22.4% Behr Paint Cooperation 12.1% Kelly Moore 2.8% Valspar 1.9% PPG Pittsburgh Paints 1.9% Zar (United Gilsonite Labs) 1.9% Devoe & Raynolds 0.9% Dutch Boy 0.9% Olympic 0.9% None of these 4.7%
Text: This statistic depicts paints used the most by U.S. construction firms in 2018 . The survey revealed that 49.5 percent of the respondents used Sherwin-Williams brand paints the most . 


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 150

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [ ]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

### Create CodeT5 (model1)

In [ ]:
load_directory = '/content/drive/MyDrive/DADS7202_final/model1-t5base-4000to6000d-100e-r1'

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100, load_best_model_at_end=True):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(load_directory)  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Year Inflation rate compared to previous year 2024* 2% 2023* 2% 2022* 2% 2021* 2.08% 2020* 2.28% 2019* 2.09% 2018 1.99% 2017 0.11% 2016 -3.08% 2015 -0.98% 2014 3.52% 2013 1.47% 2012 4.18% 2011 2.77% 2010 3.25% 2009 -0.16% 2008 5.55% 2007 2.82% 2006 3.56% 2005 3.91% 2004 1.46% 2003 1.03% 2002 -0.26% 2001 5.27% 2000 4.85% 1999 2.42% 1998 3.2% 1997 -0.01% 1996 0.93% 1995 5.86% 1994 2.66% 1993 0.78% 1992 5.66% 1991 5.54% 1990 4.33% 1989 4.4% 1988 0.83% 1987 7.04% 1986 2.17% 1985 0.5% 1984 2.05%</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'This statistic shows the average inflation rate in Saint Lucia from 1984 to 2018, with projections up until 2024. In 2018, the average inflation rate in Saint Lucia amounted to about 1.99 percent compared to the previous year.</s>'

In [ ]:
model = CodeT5()

### Train (fit) model1

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [ ]:
from transformers import TrainingArguments

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-6000to6763d-100e-r3-timeout"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)


wandb_logger = WandbLogger(name='model1-t5base-6000to6763d-100e-r3', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=5,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger,
                  #args = TrainingArguments(load_best_model_at_end=True, output_dir = "/content/drive/MyDrive/Test_output_dir", evaluation_strategy ='epoch', save_steps=200 ),  
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./CodeT5/35g3ube6/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.mod

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### check layers and weight in model

In [ ]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
#for key, value in weights.items():
    #print(key, value.shape)

In [ ]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
#print(layer_weights)

************* ตั้ง path ที่จะ save model อันแรก ***********

In [ ]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-6000to6763d-100e-r2"

In [ ]:
# save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [ ]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 115
})


In [ ]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Name (location) Generating capacity in gigawatts Three Gorges Dam (Yangtze River, China) 22.5 Itaipu Dam (Paraná River, Brazil/Paraguay) 14.0 Xiluodu (Jinsha River, China) 13.86 Guri (Caroní River, Venezuela) 10.24 Tucuruí (Tocantins River, Brazil) 8.37 Grand Coulee (Columbia River, United States) 6.81 Longtan Dam (Hongshui River, China) 6.43 Xiangjiaba (Jinsha River, China) 6.4 Krasnoyarsk (Yensei River, Russia) 6.0 Robert-Bourassa (La Grande, Canada) 5.62


In [ ]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: This statistic displays the world 's largest hydroelectric dams as of 2018 , based on generating capacity . During this year , one of the largest hydropower facilities was located along the Columbia River in Washington , United States ; it had a generating capacity of 6.81 gigawatts . Three Gorges Dam – additional informationChina is by far one of the strongest supporters of hydroelectric energy in the world . It is one of the cheapest energy facilities to maintain due to low operation costs . In the United States , the average expense for operating a hydroelectric power plant was about 6.65 mills per kilowatt hour in 2016 . As of 2018 , China accounted for 28.7 percent of global hydropower consumption . Four of the world 's top ten largest hydroelectric dams are located in China ; the Three Gorges Dam on the Yangtze River is considered the largest dam in the world and has a generating capacity of 22,500 megawatts . It is located in the Hubei province and was completed in July 20

In [ ]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

### Generate data from own model

In [ ]:
j = 5

In [ ]:
test_example = dataset2['test'][j]
print("Key:", test_example['key'])

Key: Company Name Market capitalization in million euros Intesa Sanpaolo 38043.87 UniCredit 24228.42 Mediobanca 8863.88 Fineco Bank 5902.19 UBI Banca 2947.23 Banco BPM 2842.07 BPER Banca 1935.76 Banca Monte dei Paschi di Siena 1737.76 Credito Emiliano 1640.93 Banca Popolare di Sondrio 761.8


In [ ]:
test_example = dataset2['test'][j]
print("Text:", test_example['text'])

Text: With a market capitalization of over 38 billion euros , Intesa Sanpaolo was the leading bank in Italy listed on the Milan Stock Exchange as of September 2019 . Intesa Sanpaolo , together with its fierce competitor UniCredit , dominates the banking sector in the country . The two banking groups are not only the leaders in the domestic market , but also ranked among the 20 leading banks in Europe in terms of market capitalization in 2019 . Market capitalization Market capitalization , often used as a measurement of a company 's size , is calculated by multiplying the number of outstanding shares by the current market price of one share . Market capitalization is an important metric for investors as it is simple to calculate and is also a good indicator for risk assessment . Banks are usually among the most valuable companies listed on the stock exchange . For example , Intesa Sanpaolo and UniCredit ranked third and fourth respectively among the companies listed on the Milan Stock E

In [ ]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : Intesa Sanpaolo was the leading bank in Italy in 2018, with market capitalization amounting to approximately 3.85 billion euros. The leading bank in the country was Intesa Sanpaolo, with market capitalization amounting to 3.85 billion euros. The leading bank in the country was Intesa Sanpaolo, with market capitalization amounting to 3.85 billion euros. Intesa Sanpaolo, a subsidiary of the Italian bank, was the second-largest bank in the country in 2018, with market capitalization amounting to 3.85 billion euros. Intesa Sanpaolo, the company, had a market


Create list of predict data and ref data

In [ ]:
predict = []
refer = []
for i in range(300):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [ ]:
print(len(predict))
print(len(refer))

300
300


In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

### Evaluation

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.12341329823014215, 'precisions': [0.2669437077945157, 0.14163243928660033, 0.09236456625066014, 0.06642935224374247], 'brevity_penalty': 1.0, 'length_ratio': 1.8916528754654531, 'translation_length': 36577, 'reference_length': 19336}


In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.49567227575099326, 'p': 0.41714602692382113, 'f': 0.433794892776891}, 'rouge-2': {'r': 0.27305183378161413, 'p': 0.18417814432477, 'f': 0.20911947073603726}, 'rouge-l': {'r': 0.48089763911708, 'p': 0.40252257614134734, 'f': 0.42011577555058516}}


## Model2 for IELTS dataset

### Create data from Ielt DATASET

*********** แก้ path ************

In [ ]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

In [ ]:
import pickle

with open('List_key_ielts.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load ielts key

In [ ]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [ ]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [ ]:
import pickle

with open('txt_key_ielts.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load ielts txt

In [ ]:
import pickle

with open('txt_key_ielts.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [ ]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:140] ,txt_key[:140]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[140:170] ,txt_key[140:170]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[170:] ,txt_key[170:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 140
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 30
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 31
})}
<class 'dict'>


In [ ]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 140
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 30
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 31
    })
})


**Train ielts data**

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=4)
valid_dataloader = DataLoader(dataset['validation'], batch_size=2)
test_dataloader = DataLoader(dataset['test'], batch_size=2)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
!pip install -q pytorch-lightning wandb

### Create CodeT5 (model2)

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [ ]:
load_directory = '/content/drive/MyDrive/DADS7202_final/model1-t5base-6000to6763d-100e-r1'

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100, load_best_model_at_end=True):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(load_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Airport visitors, 1995-2000 Millions of passengers per year 1995 1996 1997 1998 1999 2000 John F. Kennedy 26 35 47 35 32 44 LaGuardia 35 40 42 46 50 68</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'The bar chart gives information about how many people visited New York City through three major airports, over a six-year period between 1995 and 2000. Overall, it can be seen that over the period, there was a fluctuant trend in the number of passengers who travelled via John F. Kennedy airport, while the other two airports saw an upward trend. Another interesting point is that LaGuardia airport was the most popular at the end. Looking at the detail, the number of travellers at John F. Kennedy airport started at 26m in 1995, and then increased remarkably to reach the highest point of 47m in 1997. In 1999, the figure dipped to 32m. At the last year, there was a slight growth to 44m. On the other hand, LaGuardia began at 35m in 1995, after that it rose remarkably for the next three years, at 46m. In 2000, LaGuardia hit the peak point at 68m travellers. However, if we look at Newark airport, it started at the lowest point of 16m passengers in the first year. After this point, the trend i

Create new model to train again

In [ ]:
model = CodeT5()

### Train (fit) model2

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model2-t5base-ielts-100e-r3-timeout"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)

wandb_logger = WandbLogger(name='model2-t5base-ielts-100e-r1', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)

lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, 
                             lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [ ]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model2-t5base-ielts-100e-r3"

In [ ]:
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [ ]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 31
})


In [ ]:
test_example = dataset2['test'][1]
print("Key:", test_example['key'])

Key: unemployment rate in Asia 2012 2013 2014 Phippines 5.5 5.3 5.8 China 41 42 43 Japan 3.4 3.3 4.2 Singapore 2 1.8 1.4 India 47 5.2 46


In [ ]:
test_example = dataset2['test'][1]
print("Text:", test_example['text'])

Text: In all three years, unemployment was highest in the Philippines. The unemployment rate in the Philippines peaked at nearly 6 percent in 2014. Unemployment was second highest in India for all three years. Third came unemployment in China. Unemployment in Japan was fourth highest, and very close to China’s rate of unemployment in 2014—around 4.25 percent. Singapore had the second to lowest unemployment rates for all three years. Data was not available for Thailand in 2012, but in 2013 and 2014 it had the lowest unemployment rates of all six countries. Unemployment was at its lowest in 2014 in Thailand, when it was below 1.25 percent. For Thailand, India, and Singapore, unemployment was lowest in 2014. For the Philippines, China, and Japan, unemployment was highest in that same year.


******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

### Generate data from ielts model

In [ ]:
load_directory = '/content/drive/MyDrive/DADS7202_final/model2-t5base-ielts-100e-r2'

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(load_directory)

In [ ]:
test_example = dataset2['test'][11]
print("Key:", test_example['key'])

Key: Percentage of tourists to England who visited Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 2000 20% 35% 11% 25% 2005 15% 32% 15% 27%


In [ ]:
test_example = dataset2['test'][11]
print("Text:", test_example['text'])

Text: The line graph shows the percentage of tourists to England who visited certain Brighton attractions between 1980 and 2010. We can see that in 1980 and in 2010 the favourite attractions were the pavilion and the festival. In 1980 the least popular attraction was the pier but in 2010 this changed and the art gallery was the least popular. During the 1980s and 1990s there was a sharp increase in visitors to the pavilion from 28% to 48% and then the percentage gradually went down to 31% in 2010. The trend for the art gallery was similar to the pavilion. Visitors increased rapidly from 22% to 37% from 1980 to 1985 then gradually decreased to less than 10% over the next twenty-five years. The number of tourists who visited the Brighton Festival fluctuated slightly but in general remained steady at about 25%. Visitors to the pier also fluctuated from 1980 to 2000 then rose significantly from 12% to 22% between 2000 and 2010.


In [ ]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated by T5-base :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated by T5-base : The proportion of tourists to England who visited Brighton attractions was almost double that of the British population in 1980 and 2000. The proportion of tourists who visited Brighton attractions increased from 22% in 1980 to 23% in 2000. The proportion of tourists who visited Brighton attractions increased from 27% in 2000 to 27% in 2005. The proportion of tourists who visited Brighton attractions increased from 27% in 1980 to 23% in 2000. The proportion of tourists who visited Brighton attractions increased from 27% in 1980 to 23% in 2000. In 2000, the proportion of tourists increased from 15% to 27% in 2000. In 2000, the proportion of tourists visiting Brighton attractions increased from 15% in 1980 to 23% in 2000. In 2000, the proportion of visitors to Brighton attractions increased from 15% in 2000 to 27% in 2005. In 2000, the proportion of visitors to Brighton attractions increased from 15% in 1980 to 27% in 2005. In 2000, the proportion of visitors to Br

### Evaluation

In [ ]:
predict = []
refer = []
for i in range(31):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

NameError: ignored

In [ ]:
print(len(predict))
print(len(refer))

In [ ]:
i = 11
print(dataset2['test'][i]['key'])
print(predict[i])
print(refer[i])

Percentage of tourists to England who visited Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 2000 20% 35% 11% 25% 2005 15% 32% 15% 27%
The proportion of tourists to England who visited Brighton attractions in 1980 was 22% and 23% respectively. In the same year, the proportion of tourists stayed at Brighton attractions, Art Gallery Pavilion Pier Festival and Art Gallery Pavilion Pier Festival. The proportion of tourists who visited Brighton attractions rose, and in 2000 the proportion fell to 23%. The proportion fell again in 2000, but in 2000 the proportion fell to 23%. In 1980, 22% of tourists stayed at Brighton attractions, and in 2005 the proportion fell to 27%. In 2000, this percentage fell to 23%, and in 2000 the proportion fell to 23%. In 2000, the proportion fell to 23%, and in 2000 the proportion fell to 23%. In 2000, the proportion of tourists to Brighton attractions fell to 27%, and in 

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 9.8 MB/s 


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=[predict[11]], references=[refer[11]])
print(results)

{'bleu': 0.0, 'precisions': [0.16666666666666666, 0.03816793893129771, 0.0, 0.0], 'brevity_penalty': 0.6951439283988786, 'length_ratio': 0.7333333333333333, 'translation_length': 132, 'reference_length': 180}


In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

In [ ]:
#pip install evaluate

In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.004765372306048147, 'precisions': [0.1394899536321484, 0.02268283144309738, 0.006730007917656373, 0.002404809619238477], 'brevity_penalty': 0.3167836210898229, 'length_ratio': 0.46521660974294443, 'translation_length': 2588, 'reference_length': 5563}


In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.08141578450101047, 'p': 0.3733788399581357, 'f': 0.13002250223379141}, 'rouge-2': {'r': 0.008861952676579916, 'p': 0.06123949779135612, 'f': 0.014938932221405498}, 'rouge-l': {'r': 0.07270858947313823, 'p': 0.3414139836873509, 'f': 0.11643296917276913}}


### Test with original t5-base model

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gagan3012/k2t-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
predict = []
refer = []
for i in range(31):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.3 MB/s 


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.004765372306048147, 'precisions': [0.1394899536321484, 0.02268283144309738, 0.006730007917656373, 0.002404809619238477], 'brevity_penalty': 0.3167836210898229, 'length_ratio': 0.46521660974294443, 'translation_length': 2588, 'reference_length': 5563}


In [ ]:
test_example = dataset2['test'][0]
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
outputs = model.generate(input_ids,min_length=150, max_length=200)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Amandline Bakesry Mari Bakeshop Bolo Cakery 2001 20,000.00 80,000.00 58,000.00 2002 25,000.00 95,000.00 58,000.00 2003 21,000.00 90,000.00 58,000.00 2004 39,000.00 80,000.00 58,000.00 2005 39,000.00 80,000.00 58,000.00 2005 39,000.00 80,000.00 58,000.00 2005 39,000.00 80,000.00 58,000.00 58,000.00 2006 38,000.00 61,000.00 60,000.00 2007 41,500.00 59,000.00 66,000.00 2008 62,500.00 4 


## Check libraries version

In [ ]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset
import torch
import pickle
import transformers
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

import evaluate
import rouge
from rouge import Rouge 

In [ ]:
print('cv2 :', cv2.__version__)
print('pytesseract :', pytesseract.__version__)
#print('shutil :', shutil.__version__)
#print('os :', os.__version__)
#print('random :', random.__version__)
print('Image :', Image.__version__)
print('re :', re.__version__)
print('csv :', csv.__version__)
print('pyarrow :', pa.__version__)
#print('DatasetDict :', DatasetDict.__version__)
#print('Dataset :', Dataset.__version__)
print('torch :', torch.__version__)
print('pytorch_lightning :', pl.__version__)
#print('pickle :', pickle.__version__)
print('transformers :', transformers.__version__)
print('wandb :', wandb.__version__)
#print('sys :', sys.__version__)
#print('signal :', signal.__version__)
print('evaluate :', evaluate.__version__)
print('rouge :', rouge.__version__)

cv2 : 4.6.0
pytesseract : 0.3.10
Image : 9.3.0
re : 2.2.1
csv : 1.0
pyarrow : 9.0.0
torch : 1.13.0+cu116
pytorch_lightning : 1.8.6
transformers : 4.25.1
wandb : 0.13.7
evaluate : 0.4.0
rouge : 1.0.1


## Check layer model

In [ ]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
for key, value in weights.items():
    print(key, value.shape)

shared.weight torch.Size([32128, 768])
encoder.embed_tokens.weight torch.Size([32128, 768])
encoder.block.0.layer.0.SelfAttention.q.weight torch.Size([768, 768])
encoder.block.0.layer.0.SelfAttention.k.weight torch.Size([768, 768])
encoder.block.0.layer.0.SelfAttention.v.weight torch.Size([768, 768])
encoder.block.0.layer.0.SelfAttention.o.weight torch.Size([768, 768])
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight torch.Size([32, 12])
encoder.block.0.layer.0.layer_norm.weight torch.Size([768])
encoder.block.0.layer.1.DenseReluDense.wi.weight torch.Size([3072, 768])
encoder.block.0.layer.1.DenseReluDense.wo.weight torch.Size([768, 3072])
encoder.block.0.layer.1.layer_norm.weight torch.Size([768])
encoder.block.1.layer.0.SelfAttention.q.weight torch.Size([768, 768])
encoder.block.1.layer.0.SelfAttention.k.weight torch.Size([768, 768])
encoder.block.1.layer.0.SelfAttention.v.weight torch.Size([768, 768])
encoder.block.1.layer.0.SelfAttention.o.weight torch.Size([768

In [ ]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
#print(layer_weights)